In [ ]:
import streamlit as st
import pandas as pd
import json
import geopandas as gpd
from shapely.geometry import Point
from shapely import wkt

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()



In [ ]:
# Check what you actually have in your dataset
diagnostic = session.sql("""
    SELECT 
        COUNT(*) as total_points,
        MIN(ST_X(geometry)) as min_lon,
        MAX(ST_X(geometry)) as max_lon,
        MIN(ST_Y(geometry)) as min_lat,
        MAX(ST_Y(geometry)) as max_lat
    FROM OVERTURE_MAPS__PLACES.CARTO.PLACE
    LIMIT 100000
""").to_pandas()

print("Your dataset coverage:")
print(diagnostic)
print("\n📍 Reference:")
print("  New York: lon -79.76 to -71.86, lat 40.48 to 45.01")
print("  Sweden: lon 10 to 24, lat 55 to 69")

In [ ]:
df_station = session.sql("""
    SELECT 
        id,
        geometry,
        names,
        categories,
        ST_X(geometry) as lon,
        ST_Y(geometry) as lat
    FROM OVERTURE_MAPS__PLACES.CARTO.PLACE
    WHERE ST_X(geometry) BETWEEN -79.76 AND -71.86
      AND ST_Y(geometry) BETWEEN 40.48 AND 45.01
""").to_pandas()

print(f"Downloaded {len(df_station):,} NY records")

In [ ]:
df_station.head()

In [ ]:
import json

def extract_categories(row):
    cats = set()
    if pd.isna(row):
        return cats
    obj = json.loads(row)
    if "primary" in obj:
        cats.add(obj["primary"])
    if "alternate" in obj:
        cats.update(obj["alternate"])
    return cats

print(df_station.columns)

all_categories = set()
for row in df_station["CATEGORIES"]:
    all_categories.update(extract_categories(row))

print(all_categories)


In [ ]:
transport_categories = {
    "train_station",
    "metro_station",
    "bus_service",
    "airport_shuttles",
    "park_and_rides",
    "cable_car_service",
    "ferry_boat_company"
}

def remove_unnecessary_rows(cat_json):
    if pd.isna(cat_json):
        return set()
    obj = json.loads(cat_json)
    cats = set()
    cats.add(obj.get("primary"))
    for alt in obj.get("alternate", []):
        cats.add(alt)
    return cats
    
df_transport = df_station[df_station["CATEGORIES"].apply(
    lambda x: len(remove_unnecessary_rows(x) & transport_categories) > 0
)]

df_transport.head()

In [ ]:
import sqlite3

conn = sqlite3.connect("MY_DB_1")

df_transport.to_sql(
    "public_transport_stations",
    conn,
    if_exists="replace",
    index=False
)

conn.close()

In [ ]:
import streamlit as st
import pandas as pd
import pydeck as pdk

# -----------------------------
# App Title
# -----------------------------
st.title("🗽 New York State Public Transportation Stations")

st.write("This map visualizes all public transportation locations filtered from your dataset.")

# -----------------------------
# Map Layer
# -----------------------------
layer = pdk.Layer(
    "ScatterplotLayer",
    data=df_transport,          # make sure you're using the filtered df
    get_position="[LON, LAT]",
    get_radius=150,
    get_fill_color="[0, 100, 255]",
    pickable=True,
)

# Initial viewport centered on NY
view_state = pdk.ViewState(
    latitude=42.9,
    longitude=-75.0,
    zoom=6,
    pitch=0,
)

# Render map
r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    tooltip={"text": "{primary}"},
)

st.pydeck_chart(r)

# Show raw data
with st.expander("Show Raw Data"):
    st.dataframe(df_transport)


In [ ]:
session.write_pandas(df_transport, "MY_COORDS", auto_create_table=True)

In [ ]:
session.get_current_database(), session.get_current_schema()

In [ ]:
SELECT 
    ZIP_CODE,
    ST_SRID(GEOMETRY) AS srid,
    ST_ASWKT(GEOMETRY) AS wkt_sample
FROM U_S__ZIP_CODE_METADATA_WITH_GEOMETRY.PUBLIC.ZIP_CODE_GEOMETRY_SHARE
LIMIT 5;

In [ ]:
-- SELECT
--     c.*,
--     z.ZIP_CODE,
--     z.STATE
-- FROM MY_DB_1.PUBLIC.MY_COORDS c
-- LEFT JOIN U_S__ZIP_CODE_METADATA_WITH_GEOMETRY.PUBLIC.ZIP_CODE_GEOMETRY_SHARE z
--     ON ST_CONTAINS(
--         TO_GEOGRAPHY(ST_SETSRID(z.GEOMETRY, 4326)),
--         TO_GEOGRAPHY('POINT(' || c.LON || ' ' || c.LAT || ')')
--     );

SELECT
    c.*,
    z.ZIP_CODE,
    z.STATE
FROM MY_DB_1.PUBLIC.MY_COORDS c
LEFT JOIN U_S__ZIP_CODE_METADATA_WITH_GEOMETRY.PUBLIC.ZIP_CODE_GEOMETRY_SHARE z
    ON ST_DWITHIN(
        TO_GEOGRAPHY(ST_SETSRID(z.GEOMETRY, 4326)),
        TO_GEOGRAPHY('POINT(' || c.LON || ' ' || c.LAT || ')'),
        200  -- distance in meters
    );



In [ ]:
CREATE OR REPLACE TABLE MY_DB_1.PUBLIC.MY_COORDS_WITH_ZIP AS
SELECT
    c.*,
    z.ZIP_CODE,
    z.STATE
FROM MY_DB_1.PUBLIC.MY_COORDS c
LEFT JOIN U_S__ZIP_CODE_METADATA_WITH_GEOMETRY.PUBLIC.ZIP_CODE_GEOMETRY_SHARE z
    ON ST_CONTAINS(
        TO_GEOGRAPHY(ST_SETSRID(z.GEOMETRY, 4326)),
        TO_GEOGRAPHY('POINT(' || c.LON || ' ' || c.LAT || ')')
    );


In [ ]:
CREATE OR REPLACE TABLE MY_DB_1.PUBLIC.COORDS_BY_ZIP AS
SELECT
    ZIP_CODE,
    STATE,
    COUNT(*) AS num_points
FROM MY_DB_1.PUBLIC.MY_COORDS_WITH_ZIP
WHERE ZIP_CODE IS NOT NULL
GROUP BY ZIP_CODE, STATE
ORDER BY num_points DESC;


In [ ]:
SELECT * 
FROM MY_DB_1.PUBLIC.COORDS_BY_ZIP
LIMIT 50;

In [ ]:
CREATE OR REPLACE TABLE MY_DB_1.PUBLIC.COORDS_BY_ZIP_NY AS
SELECT *
FROM MY_DB_1.PUBLIC.COORDS_BY_ZIP
WHERE STATE = 'NY';

In [ ]:
CREATE OR REPLACE TABLE MY_DB_1.PUBLIC.LOCATION_VULNERABILITY_NUMERIC AS
WITH combined AS (
    SELECT
        COALESCE(c.ZIP_CODE, d.ZIP_CODE) AS ZIP_CODE,
        COALESCE(c.num_points, 0) AS num_transport_stations,
        COALESCE(d.TOTAL_PROVIDERS, 0) AS total_facilities,
        COALESCE(c.num_points, 0) + COALESCE(d.TOTAL_PROVIDERS, 0) AS total_count
    FROM MY_DB_1.PUBLIC.COORDS_BY_ZIP c
    FULL OUTER JOIN MY_DB_1.PUBLIC.DIA_ZIP_CODE d
        ON c.ZIP_CODE = d.ZIP_CODE
    WHERE c.STATE = 'NY' OR c.STATE IS NULL  -- Keep only NY coordinates
),
stats AS (
    SELECT
        MIN(total_count) AS min_count,
        MAX(total_count) AS max_count
    FROM combined
)
SELECT
    ZIP_CODE,
    num_transport_stations,
    total_facilities,
    total_count AS total_facilities_and_transportation_stations,
    -- numeric vulnerability: normalized inverse of total_count
    ROUND(1 - (total_count - stats.min_count) / NULLIF(stats.max_count - stats.min_count, 0), 3) AS location_vulnerability_score
FROM combined
CROSS JOIN stats
ORDER BY location_vulnerability_score DESC;


In [ ]:
SELECT * 
FROM MY_DB_1.PUBLIC.LOCATION_VULNERABILITY_NUMERIC 
LIMIT 50;